# Classes and Methods

## - The ```PathConstructor``` class

In [2]:
import operator as op
from functools import reduce

In [3]:
class PathConstructor:
    """Class to construct and count m-paths in a n**d grid """
    
    def __init__(self,d,m,n,start_coords):
        """Initialize class 
        Input:
            - d = no of dimensions
            - m = no of unit steps taken in each path
            - n = no of grid lines
            - start_coords = coords of the starting position 
        """
        
        #attributes 
        self.dimensions = d
        self.grid_size = n
        self.max_path_size = m
        self.coords = start_coords
        
        #count the number of all possible 1D paths along each dimension for
        #the given starting position, and store it in a dictionary
        self.storage = dict()  
        for coord in range(d):
            for k in range(0,m+1):
                self.storage[(coord,k)] = len(self.one_dim_paths(coord,no_steps=k))
        
        
    #------------------ Public Methods --------------------
    
    #1) Constructing Paths
        
    def one_dim_paths(self,coord,no_steps):
        """Method to construct all possible 1D paths
        Input:
            - coord = coordinate-dimension; must lie in [0,d)
            - no_steps = no of unit steps taken in each path
        Output:
            a list of all possible 1D paths
        """

        initial_paths = [[[self.coords[coord]]]]
        paths = self._find_paths(no_steps=no_steps,current_paths=initial_paths,dimensions=1)
        
        return paths
    
    def find_max_paths(self):
        """Method to construct all possible d-dimensional paths
        Output:
            a list of all d-dimensional possible paths
        """
        
        return self._find_paths(dimensions=self.dimensions,
                               no_steps=self.max_path_size,
                               current_paths=[[self.coords]])
    
    #2) Counting Paths
    
    def path_counter(self):
        """Method to count all possible d-dimensional paths by brute-force, 
        i.e. by constructing and counting all d-dimensional paths.
        """
        
        return len(self.find_max_paths())
    
    
    def comb_counter(self,dimensions,no_steps):
        """Method to count all possible d-dimensional paths by recursively counting 
        all possible k-dimensional path combinations"""
    
        if dimensions==1:
            return self.storage[(0,no_steps)]

        else:
            return sum([self._ncr(no_steps,k) 
                        * self.comb_counter(dimensions-1,k) 
                        * self.storage[(dimensions-1,no_steps-k)] 
                        for k in range(0,no_steps+1)]
                      )



    
        
    #------------------ Internal Methods --------------------
        
    def _ncr(self,n, r):
        """Method to compute combinatorics."""
        
        r = min(r, n-r)
        numer = reduce(op.mul, range(n, n-r, -1), 1)
        denom = reduce(op.mul, range(1, r+1), 1)
        
        return int(numer / denom)

    def _find_next_walks(self,dimensions,start):
        """Method to compute all possible unit-step walks in d dimensions, 
        starting from coordinates 'start'."""
    
        next_walks = []

        #iterate over possible dimensions
        for i in range(dimensions):

            #if walk is within valid range append coords of walk to next_walks 
            if 0 <= start[i]+1 < self.grid_size:
                walk = start.copy()
                walk[i]+=1
                next_walks.append(walk)

            if 0 <= start[i]-1 < self.grid_size:
                walk = start.copy()
                walk[i]-=1
                next_walks.append(walk)

        return next_walks
    
    def _find_paths(self,dimensions,no_steps,current_paths,current_step=0):
        """Method to recursively construct all possible paths.
        """
    
        #stop recursion when reach max steps
        if current_step == no_steps:
            return current_paths

        # for each current path find all possible walks and hence all new paths
        new_paths = [] 
        for path in current_paths:
            start = path[current_step]
            next_walks = self._find_next_walks(dimensions,start)

            for walk in next_walks:
                new_paths.append(path+[walk])

        return self._find_paths(dimensions,no_steps,current_paths=new_paths,current_step=current_step+1)
    

## - Helper Functions

In [21]:
from itertools import product, combinations_with_replacement

In [22]:
def start_coords_generator(d,n):
    """Function to generate a dictionary of all coordinate points in a n**d-grid. 
    Points equivalent under certain symmetries are grouped together 
    as a key,value pair in a dictionary, with 
    key = a coord point representive of the equivalence classes
    value = multiplicity (dimension of equivalence class)
    """
    
    coords_generator = map(''.join, product(''.join([str(i) for i in range(n)]), repeat=d))
    
    symmetric_coords = {}
    for start_coords in coords_generator:
        l = [int(coord) for coord in start_coords]
        l.sort()
        key = tuple(l)
        symmetric_coords[key] = symmetric_coords.get(key,0) + 1
        
    return symmetric_coords

In [23]:
def generate_unique_coords(d,n):
    """Method to generate all the coord point representatives of 
    the equivalence classes obtained by symmetries. 
    Note that these points lie in the region 
    0<= x_{d-1} <= x_{d-2} <= x_0 
    defined by the intersection of all symmetric hyperplanes."""
    
    mid = (n-1)//2
    return combinations_with_replacement(list(range(mid+1)),d)

## Stress Tests:

## - Class methods

### Test 1:  Recreate the given example

In [6]:
d=2
m=2
n=3
start_coords=[0]*d
pc = PathConstructor(d,m,n,start_coords)

In [8]:
pc.find_max_paths()

[[[0, 0], [1, 0], [2, 0]],
 [[0, 0], [1, 0], [0, 0]],
 [[0, 0], [1, 0], [1, 1]],
 [[0, 0], [0, 1], [1, 1]],
 [[0, 0], [0, 1], [0, 2]],
 [[0, 0], [0, 1], [0, 0]]]

In [7]:
pc.path_counter()

6

### Test 2: Counting Paths Methods

In [11]:
d=4
m=7
n=10
start_coords=[0]*d
pc = PathConstructor(d,m,n,start_coords)

In [15]:
pc.comb_counter(dimensions=d,no_steps=m) == pc.path_counter()

True

In [12]:
#fast method
%timeit pc.comb_counter(dimensions=d,no_steps=m)

428 µs ± 12.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [13]:
#slow method
%timeit pc.path_counter()

206 ms ± 7.27 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## - Coord generators

Check if ```start_coords_generator``` method computes the correct multiplicities for each equivalence class

In [20]:
d,n = 4,8
sum([v for k,v in start_coords_generator(d,n).items()]) == (n**d)

True

# Questions

## Q1:
Consider the case where d=4, n=10, and m=10.

In [54]:
from numpy import std,mean
d,n,m = (4,10,10)

a) How many valid walks start from the corner (0, 0, 0, 0)?

In [55]:
start_coords=[0]*d
PathConstructor(d,m,n,start_coords).comb_counter(dimensions=d,no_steps=m)

44569724

b) Consider the count of valid walks associated with each possible starting position. What is the ratio of the highest count of valid walks to smallest count of valid walks?

In [63]:
#NOTE:
all_coords = n**d
symmetric_coords = len(start_coords_generator(d,n))
print(f'{symmetric_coords}/{all_coords}')
print('Pct reduction: {:2f}%'.format((1-symmetric_coords/all_coords)*100))

715/10000
Pct reduction: 92.850000%


In [57]:
def create_all_counts(d,n,m):
    all_counts=[]
    for start_coords,permutations in start_coords_generator(d,n).items():
        pc = PathConstructor(d,m,n,list(start_coords))
        path_counts = pc.comb_counter(dimensions=d,no_steps=m)
        for i in range(permutations):
            all_counts.append(path_counts)
    
    return all_counts


In [58]:
all_counts = create_all_counts(d,n,m)

In [59]:
#check if dimensions of all_counts is correct
n**d,len(all_counts),len(start_coords_generator(d,n))

(10000, 10000, 715)

In [60]:
max(all_counts)/min(all_counts)

23.81209118548726

c) Consider the count of valid walks associated with each possible starting position. What is the ratio of the standard deviation of the number of valid walks to the mean of the number of valid walks?

In [61]:
std(all_counts)/mean(all_counts)

0.5106573744484553

## Q2:
Consider the case where d=8,n=12 and m=12.

In [64]:
d,n,m = (8,12,12)

a) How many valid walks start from the corner (0, 0, 0, 0)?

In [49]:
start_coords=[0]*d
PathConstructor(d,m,n,start_coords).comb_counter(dimensions=d,no_steps=m)

2479092118264

b) Consider the count of valid walks associated with each possible starting position. What is the ratio of the highest count of valid walks to smallest count of valid walks?

In [65]:
# NOTE:
unique_coords = len(list(generate_unique_coords(d,n)))
all_coords = n**d
print(f'{unique_coords}/{all_coords}')
print('Pct reduction: {:2f}%'.format((1-unique_coords/all_coords)*100))

1287/429981696
Pct reduction: 99.999701%


In [6]:
def create_unique_counts(d,n,m):
    unique_counts = []
    for start_coords in generate_unique_coords(d,n):
        path_counts = PathConstructor(d,m,n,list(start_coords)).comb_counter(dimensions=d,no_steps=m)
        unique_counts.append(path_counts)
        
    return unique_counts



In [7]:
unique_counts = create_unique_counts(d,n,m)

In [8]:
max(unique_counts)/min(unique_counts)

113.51216384122472

c) Consider the count of valid walks associated with each possible starting position. What is the ratio of the standard deviation of the number of valid walks to the mean of the number of valid walks?

In [66]:
#Need to generate multiplicites of all unique coord points